In [1]:
from IPython.display import clear_output

In [2]:
!pip3 install -r requirements.txt
clear_output()

In [3]:
import feedparser
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import matplotlib
import plotly.express as px
import plotly.graph_objects as go

In [4]:
class Parser:
    """ Parses out rss feeds"""

    def __init__(self, start_date='2022-10-01', end_date='2022-11-01'):
        
        self.start_date = datetime.strptime(start_date, '%Y-%m-%d')
        self.end_date = datetime.strptime(end_date, '%Y-%m-%d')

    def retrieve_rss_feed(self, url):
        xml_data = feedparser.parse(url)
        return pd.DataFrame(xml_data.entries)
            

In [5]:
ps = Parser('2022-10-01', '2022-11-01')

In [6]:
#Crime Junkie rss
cj_df = ps.retrieve_rss_feed('https://feeds.simplecast.com/qm_9xx0g')

In [7]:
cj_df['CJ_text_to_compare'] = cj_df.title.str.split(':')
cj_df['CJ_text_to_compare'] = cj_df['CJ_text_to_compare'].apply(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cj_df['CJ_text_to_compare'] = cj_df['CJ_text_to_compare'].str.replace('[0-9]:;",-', '', regex=True)
cj_df['CJ_text_to_compare'] = cj_df['CJ_text_to_compare'].str.replace('Part', '', regex=False)

In [8]:
# Morbid rss
morbid_df = ps.retrieve_rss_feed('https://rss.art19.com/morbid-a-true-crime-podcast')

In [9]:
morbid_df.head()

,title,title_detail,summary,summary_detail,itunes_title,itunes_episodetype,itunes_episode,content,id,guidislink,published,published_parsed,itunes_explicit,image,tags,itunes_duration,links
0,Episode 389: The Murder of Ali Kamel Fahmy Bey,"{'type': 'text/plain', 'language': None, 'base...",<p>Marguerite Alibert was a woman set on getti...,"{'type': 'text/html', 'language': None, 'base'...",The Murder of Ali Kamel Fahmy Bey,full,389,"[{'type': 'text/plain', 'language': None, 'bas...",gid://art19-episode-locator/V0/UK3_zsKk7oXX9I8...,False,"Wed, 09 Nov 2022 08:00:00 -0000","(2022, 11, 9, 8, 0, 0, 2, 313, 0)",True,{'href': 'https://content.production.cdn.art19...,"[{'term': 'True crime', 'scheme': 'http://www....",01:38:54,"[{'type': 'audio/mpeg', 'length': '94958236', ..."
1,Episode 388: Halloween & The Original Hellfire...,"{'type': 'text/plain', 'language': None, 'base...",<p>Today we have the esteemed pleasure of sitt...,"{'type': 'text/html', 'language': None, 'base'...",Halloween & The Original Hellfire Club with Ca...,full,388,"[{'type': 'text/plain', 'language': None, 'bas...",gid://art19-episode-locator/V0/E_SkxUrVc3hrTgF...,False,"Mon, 07 Nov 2022 08:00:00 -0000","(2022, 11, 7, 8, 0, 0, 0, 311, 0)",True,{'href': 'https://content.production.cdn.art19...,"[{'term': 'True crime', 'scheme': 'http://www....",01:10:45,"[{'type': 'audio/mpeg', 'length': '67923800', ..."
2,Episode 387: Listener Tales 56,"{'type': 'text/plain', 'language': None, 'base...",<p>We’ve got another themed Listener Tales epi...,"{'type': 'text/html', 'language': None, 'base'...",Listener Tales 56,full,387,"[{'type': 'text/plain', 'language': None, 'bas...",gid://art19-episode-locator/V0/kb4vJosf6MGm61l...,False,"Fri, 04 Nov 2022 07:00:00 -0000","(2022, 11, 4, 7, 0, 0, 4, 308, 0)",True,{'href': 'https://content.production.cdn.art19...,"[{'term': 'True crime', 'scheme': 'http://www....",01:19:13,"[{'type': 'audio/mpeg', 'length': '76056032', ..."
3,"Episode 386: Fairy Lights, Witches & Curses OH...","{'type': 'text/plain', 'language': None, 'base...",<p>Urban legends for Halloween? YOU BETCHA! A...,"{'type': 'text/html', 'language': None, 'base'...","Fairy Lights, Witches & Curses OH MY!",full,386,"[{'type': 'text/plain', 'language': None, 'bas...",gid://art19-episode-locator/V0/0co6K5mS3SXuIw4...,False,"Wed, 02 Nov 2022 07:00:00 -0000","(2022, 11, 2, 7, 0, 0, 2, 306, 0)",True,{'href': 'https://content.production.cdn.art19...,"[{'term': 'True Crime', 'scheme': 'http://www....",01:03:37,"[{'type': 'audio/mpeg', 'length': '61078047', ..."
4,Episode 385: Terrifying HometownTales with Two...,"{'type': 'text/plain', 'language': None, 'base...",<p>Friends! We finally got to sit down with Co...,"{'type': 'text/html', 'language': None, 'base'...",Terrifying HometownTales with Two Girls One Gh...,full,385,"[{'type': 'text/plain', 'language': None, 'bas...",gid://art19-episode-locator/V0/SSUAvEZapHC0NYO...,False,"Mon, 31 Oct 2022 07:00:00 -0000","(2022, 10, 31, 7, 0, 0, 0, 304, 0)",True,{'href': 'https://content.production.cdn.art19...,"[{'term': 'True Crime', 'scheme': 'http://www....",02:09:20,"[{'type': 'audio/mpeg', 'length': '124162716',..."


In [10]:
morbid_df['Morbid_text_to_compare'] = morbid_df.title.str.split(':')
morbid_df['Morbid_text_to_compare'] = morbid_df['Morbid_text_to_compare'].apply(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
morbid_df['Morbid_text_to_compare'] = morbid_df['Morbid_text_to_compare'].str.replace('[0-9]:;",-', '', regex=True)
morbid_df['Morbid_text_to_compare'] = morbid_df['Morbid_text_to_compare'].str.replace('Part', '', regex=False)

In [11]:
# Getting the text to compare into lists
cj_titles = cj_df.CJ_text_to_compare.tolist()
morbid_titles = morbid_df.Morbid_text_to_compare.tolist()

In [14]:
# Need to use multiprocessing to compare podcasts

# Testing matches
potential_matches = []
for i in cj_titles:
    potential_matches.append((i, process.extract(i, morbid_titles, limit=3, scorer=fuzz.WRatio)))

# First time comparing 285 x 449 titles took 42 seconds

In [15]:
good_matches = []
for pm in potential_matches:
    for t_matches in pm[1]:
        # This is the threshold I want
        if t_matches[1] > 88:
            good_matches.append({'CJ_Title' : pm[0],
                                'Morbid_Title' : t_matches[0],
                                'Score' : t_matches[1]
                                }   
            )
            

In [16]:
matches_df = pd.DataFrame(good_matches)

In [17]:
matches_df.head()

,CJ_Title,Morbid_Title,Score
0,Polly Klaas,The Abduction and Murder of Polly Klaas,90
1,Tara Calico,Tara Calico,100
2,Phoebe Handsjuk,The Mysterious Death of Phoebe Handsjuk 2,90
3,Phoebe Handsjuk,The Mysterious Death Of Phoebe Handsjuk 1,90
4,Brittanee Drexel,**AD FREE** The Mysterious Disappearance of Br...,90


In [18]:
joined_df = matches_df.merge(cj_df[['CJ_text_to_compare', 'published']], how='left', left_on='CJ_Title', right_on='CJ_text_to_compare')
joined_df = joined_df.merge(morbid_df[['Morbid_text_to_compare', 'published']], how='left', left_on='Morbid_Title', right_on='Morbid_text_to_compare')

In [19]:
joined_df.rename(columns={
    'published_x' : 'CJ_Date',
    'published_y' : 'Morbid_Date'
    }, inplace=True)

In [20]:
joined_df['CJ_Date'] = pd.to_datetime(joined_df['CJ_Date'])
joined_df['Morbid_Date'] = pd.to_datetime(joined_df['Morbid_Date'])

In [35]:
# Building the scatterplot
# Need to research a better way perhaps
fig = go.Figure()
fig.add_trace(go.Scatter(mode='markers',
            x=joined_df["CJ_Date"], 
            y=joined_df["CJ_Title"], 
            marker=dict(
            color='Blue'
            ),
            name='Crime Junkie'

    )
)
fig.add_trace(go.Scatter(mode='markers',
            x=joined_df["Morbid_Date"], 
            y=joined_df["CJ_Title"], 
            marker=dict(
            color='Red'
            ),
            name='Morbid'
    )
)
fig.show()

In [61]:
# Setting a flag for which podcast released first and the time difference
joined_df['CJ_First'] = joined_df['CJ_Date'] < joined_df['Morbid_Date']
joined_df['Time_Diff'] = joined_df['CJ_Date'] - joined_df['Morbid_Date']
joined_df['Abs_Time_Diff'] = joined_df['Time_Diff'].apply(lambda x: abs(x.days))

In [62]:
fig1 = px.bar(joined_df, x='CJ_First')
fig1.show()

In [64]:
time_chart = px.bar(joined_df, 
                    x='CJ_Title',
                    y='Abs_Time_Diff',
                    color='CJ_First')
time_chart.show()